In [1]:
import pdfminer 
from pdfminer.high_level import extract_text
from pdfminer.high_level import extract_text_to_fp
import os

# txt = extract_text('May 26, 2020.pdf')

In [35]:
pdf_txt_dict = {}
for fl_name in os.listdir('duterte_pdf'):
    try:
#         pdf_txt_lt.append(extract_text(f'duterte_pdf\\{fl_name}'))
        pdf_txt_dict[fl_name] = extract_text(f'duterte_pdf\\{fl_name}')
    except:
        print(fl_name)

April 25, 2018.pdf


>I found that in most PDF it follows a particular structure. The main body of the speech is right after a particular pattern which is the Location and Date of the speech  enclosed by brackets like [Malago Clubhouse, Malacañang Park, Manila | 26 May 2020] then followed by the transcript. So I used regular expression to find the first occurence of this pattern and extract everything below it.

>In my obervation there are 2 types of transcript. The first one is when it only contains President Duterte's dialogue second is when there are dialogue from other people.  So I have extract only President Duterte's dialogue. For this type of transcript I have to convert the PDF file to word document for me to determine which are President Duterte's dialogue. 

***

# 1.Some PDF has the pattern but we can't detect it for some fucking reason

>Now we're going to trimmed the top portion which the title, date, and locaiton of the transcript.

In [221]:
import re
regex = re.compile('\\[(.*?)\]')

In [230]:
pdf_txt_dict_trimmed = {}
for key in pdf_txt_dict.keys():
    try:
        span = re.search(regex, pdf_txt_dict[key].replace('\n', '')).span()
        pdf_txt_dict_trimmed[key] = pdf_txt_dict[key].replace('\n', '')[span[1]:]
    except:
        pdf_txt_dict_trimmed[key] = pdf_txt_dict[key].replace('\n', '')
        print(key)

June 22, 2019.pdf
May 21, 2019.pdf
September 4, 2018.pdf


In [233]:
pdf_txt_dict_trimmed ['March 11, 2020.pdf']

' Kindly sit down as I deliver a short speech. I’ve been told to --- masyado naman takot itong corona na ito. They are discouraging long meetings and large congregation. Naniwala pala kayo. Sus. You know if I heard correctly, the health minister of Japan and Abe tested positive for... Interior and Local Government Secretary Eduardo Año and other members of the Cabinet; Senator Bong Go; our newly promoted officers of the Armed Forces of the Philippines and Philippine National Police; other distinguished guests; ladies and gentlemen It is an honor to stand here today as I speak before the most valiant men and women in uniform in our country. First, I congratulate all of you for adding another accolade in your achievements as officials of the Armed Forces of the Philippines and the Philippine National Police. With the many challenges that the world is facing today, we need soldiers like you who are always ready to fight for our motherland’s sovereignty, defend the honor of our country, an

>May 21, 2019 and September 2018 does not have this pattern so we're going to process this manually. As for June 22, 2019 it is not actually a speech but a press release between Philippines and Indonesia.

# 1. PROBLEM: It doesn't detect it because it contains new line character 
# 1. SOLUTION: remove breakline character before stripping it.

***

# 2. PROBLEM: Two speech does not have the pattern

# 2. SOLUTION: Process this two individually
***

# 3. Some PDFs contains dialogue of another person

> Unlike an html where we can identify if there is a conversation by searching for strong tag. In PDF there is no such thing, so for us to know if there is a conversation is if it contains one of these strings 'PRESIDENT RODRIGO DUTERTE:', 'PRESIDENT DUTERTE:', 'PRESIDENT RODRIGO ROA DUTERTE:' after trimming it. Then we're probably going to convert these PDF to .docx for us to extract only President Duterte's dialogue.

>So we're going to traverse through transcirpts and identify which contains dialogue of another person.

In [271]:
lt_names= ['PRESIDENT RODRIGO DUTERTE:', 'PRESIDENT DUTERTE:', 'PRESIDENT RODRIGO ROA DUTERTE:']

In [251]:
lt_docx = []
for key in pdf_txt_dict_trimmed.keys():
    if any(name in pdf_txt_dict_trimmed[key] for name in lt):
        lt_docx.append(key)     
print(lt_docx)

['April 13, 2018.pdf', 'April 13, 2020.pdf', 'April 29, 2018.pdf', 'April 8, 2020.pdf', 'April 9, 2018.pdf', 'August 20, 2019.pdf', 'August 30, 2019.pdf', 'December 10, 2019.pdf', 'December 26, 2018.pdf', 'December 4, 2019.pdf', 'December 5, 2019.pdf', 'July 28, 2019.pdf', 'June 2, 2018.pdf', 'June 8, 2019.pdf', 'March 7, 2019.pdf', 'May 21, 2019.pdf', 'May 28, 2020.pdf', 'November 1, 2018.pdf', 'November 15, 2018.pdf', 'November 20, 2018.pdf', 'November 25, 2019.pdf', 'November 6, 2018.pdf', 'October 1, 2019.pdf', 'October 18, 2019.pdf', 'October 2, 2019.pdf', 'October 3, 2019.pdf', 'October 31, 2019.pdf', 'October 6, 2019.pdf', 'September 11, 2018.pdf', 'September 13, 2018.pdf', 'September 16, 2018.pdf', 'September 17, 2018.pdf', 'September 18, 2018.pdf', 'September 2, 2018.pdf']


>So these are the pdfs that contains dialogue of multiple people and not only President Duterte. I converted these manually to docx file and stored them to separate directory.

# 3. PROBLEM: How to identify these PDFs
# 3. SOLUTION: Check document contains full name of duterte in capslock followed by ':' (if only a few process individually)

***

# NOW WE'RE GOING TO EXTRACT DIALOGUE OF DUTERTE FROM THOSE DOCXS FILE

In [ ]:
import docx

transcript_docx = {}

#traverse through different pdf
for fl in lt_docx:
    
    doc = docx.Document(f'duterte_docx\\{fl[:-4]}.docx')
    encountered = False
    isDuterte = True
    trans = []
    
    #tranverse through the context of transcript
    for paragraph in doc.paragraphs:
        if encountered:
            for run in paragraph.runs:
                # if the text is bold we ask if it is a name of duterte or not
                if run.bold:
                    if any(nm in paragraph.text for nm in lt_names):
                        isDuterte = True
                    else:
                        isDuterte = False
            
            #we'll only include President Duterte's dialogue in the list
            if isDuterte:
                if paragraph.text:
                    trans.append(paragraph.text)


        #if we encounter the pattern except 
        if re.search(regex, paragraph.text):
            encountered = True
            
    
    single_str = ' '.join(trans)
    
    for nm in lt_names:
        single_str = single_str.replace(nm, ' ')
        
    transcript_docx[fl] = single_str 

# 3. PROBLEM 2:How to extract only duterte's part
# 3. SOLUTION 2: converte to docx and check if in strong if yes check if duterte if yes extract everything until encoutering strong again then repeat step?

***

# We'll delete May 21, 2019 and September 4, 2018 in both dictionaries because they don't contain the pattern, so we'll process it to manually.

In [370]:
del pdf_txt_dict_trimmed['May 21, 2019.pdf']
del transcript_docx['May 21, 2019.pdf']
del pdf_txt_dict_trimmed['September 4, 2018.pdf']

>We'll merge these two dictionary and store them to all_transcript_from_pdf

In [384]:
transcript_from_pdf = pdf_txt_dict_trimmed
transcript_from_pdf.update(pdf_txt_dict_trimmed)

In [386]:
list(transcript_from_pdf.keys())[:10]

['April 10, 2018.pdf',
 'April 11, 2019.pdf',
 'April 12, 2018.pdf',
 'April 13, 2018.pdf',
 'April 13, 2019.pdf',
 'April 13, 2020.pdf',
 'April 15, 2018.pdf',
 'April 16, 2019.pdf',
 'April 16, 2020.pdf',
 'April 17, 2018.pdf']

>Let's remove .pdf

In [388]:
all_transcript_from_pdf = {}
for key, val in zip(transcript_from_pdf.keys(), transcript_from_pdf.values()):
    all_transcript_from_pdf[key[:-4]] = val

In [398]:
list(all_transcript_from_pdf.keys())[:10]

['April 10, 2018',
 'April 11, 2019',
 'April 12, 2018',
 'April 13, 2018',
 'April 13, 2019',
 'April 13, 2020',
 'April 15, 2018',
 'April 16, 2019',
 'April 16, 2020',
 'April 17, 2018']

>Now we'll add the May 21, 2019 and September 4,2018

In [ ]:
with open('duterte_pdf\\May 21, 2019.txt', 'r') as fl:
    my_21_19 = fl.read()

with open('duterte_pdf\\September 4, 2018.txt', 'r') as fl:
    sp_04_18 = fl.read()

all_transcript_from_pdf['May 21, 2019'] = my_21_19.replace('\n', ' ')
all_transcript_from_pdf['September 4, 2018'] = sp_04_18.replace('\n', ' ')

In [400]:
all_transcript_from_pdf['May 21, 2019']

'Your Excellency, I was informed that youâ€™re already here. Pardon for about two minutes late. I was just signing some papers. But I am very happy of your presence today and congratulations to your new deployment. I understand this is the first ambassadorial assignment that you have for the Republic of Thailand. Iâ€™m sure that you would learn so many things along the way. You wonâ€™t have a hard time. We are similar in all things including our desire to have a peaceful southeast Pacific region. Welcome. I accept your lettre de crÃ©ance and just a few minutes may I invite you to a room so we can have a little tÃªte-Ã\xa0-tÃªte and before that Iâ€™d like to have a picture with you. Iâ€™d learned that a very handsome guy from Thailand visited my country. I accept your assignment. Can we have a picture taking?'

In [401]:
all_transcript_from_pdf['September 4, 2018']

'Well I would like to thank Israel, the government and the people for inviting us here to see for ourselves in visuals what we have read time and again in our history books. We know the brutal and cruel journey of the Israelites. From one scattered generation finally finding their place under the sun. We are happy that â€” witnesses to this event. And I said my daughter is... I wouldnâ€™t say that now. But she is a descendant of a Jew in the Philippines, probably came there to seek the sanctuary. There was a time when we welcomed Jews in the Philippines to provide them a country of refuge. I cannot speak for my daughter how she feels now but she is one of those generations that came after. Me, I realized that war is insanity. And what happened here, in Europe especially under the Nazi. I could not imagine of a country obey an insane leader. And I could not ever fathom the spectacle of a human being going into a killing spree, murdering old men, women, men, children, mother. I hope that

# Add to DataFrame

> load the file

In [402]:
import pandas as pd

df = pd.read_csv('Duterte_Speech.csv', index_col ='Unnamed: 0')

In [405]:
from_pdf_ser = pd.Series(all_transcript_from_pdf)

In [406]:
from_pdf_ser

April 10, 2018            His Excellency  President Xi Jinping and ...
April 11, 2019          Maayong gabi-i.   Te may ara man ko nga sal...
April 12, 2018        Salamat po. Thank you for your courtesy. I’d ...
April 13, 2018        Kindly sit down. Thank you. Let me just annou...
April 13, 2019            Intawon pud ang akong sundalo nga securit...
                                           ...                        
September 7, 2019       Daghang salamat sa inyong tanan. Panglingko...
September 8, 2018         Thank you. Maupo… [crowd: Duterte! Dutert...
September 9, 2019         Your Excellency, I warmly welcome you and...
May 21, 2019         Your Excellency, I was informed that youâ€™re ...
September 4, 2018    Well I would like to thank Israel, the governm...
Length: 297, dtype: object

In [517]:
df_2.index.name = 'date'

In [519]:
df_2.to_csv('Duterte_Speech2.csv')

>Now that we have every transcript let's merge them into our data frame

In [455]:
df_2 = pd.merge(df.reset_index(), temp_sr.to_frame().T.reset_index(), how='outer').set_index('index')

In [456]:
for key, val in zip(all_transcript_from_pdf.keys(), all_transcript_from_pdf.values()):
    df_2.loc[key]['transcript'] = val

>So far we have one transcript that does not work which is the speech on April 25, 2018(the pdf url does not work), and we also have a transcript that does belong to duterte which is on June 22, 2019. So we'll remove this from our dataframe.

In [530]:
df_final= df_2.drop(index=['June 22, 2019', 'April 25, 2018'])

In [535]:
df_final.shape

(314, 5)

# PREPROCESS THE TRANSCRIPT

In [560]:
df_final.iloc[0].transcript

'     So ako pati si Bong during my mayorship days... So at that moment was si Bong ang… Sabi niya, “Ako na lang muna.” Sabi niya, “Mayor, kung wala kang ibang makuha kasi…” Right there and then nasabi ko somebody should call the funeral parlor. So siya ‘yun lahat until he became… Siya ‘yung kasama ko.   So in and out kami sa --- in and out kami sa gubat. And as a matter of fact, Colonel Lorenzana was there. We’re friends. Kumakain ako sa kampo niya. But at times, galing ako sa bukid nakikiusap ako sa mga NPA.   Ako naman hindi ako nag --- I’m not trying to really pull my own chair but for all ‘yung naka --- nung mga sundalo pati pulis na nabihag ng --- except si ‘yung general --- lahat ‘yan ako ang kumukuha pati si Bong. Kaya bilib ako. Saludo ako kung ginawa ninyo.   I’m not paying you. I know that you are not mercenaries. You do not do it for money. You do it out of passion for the love of country because we are all Filipinos and we know where to place ourselves when the time comes 

In [4]:
import re

def clean_transcript(single_str):
    lt_names= ['PRESIDENT RODRIGO DUTERTE:', 'PRESIDENT DUTERTE:', 'PRESIDENT RODRIGO ROA DUTERTE:']
    str_cleaned = single_str
    for nm in lt_names:
        str_cleaned = str_cleaned.replace(nm, '')
    str_cleaned = str_cleaned.replace('\xa0', '')
    str_cleaned = str_cleaned.replace('\x0c', '')
    str_cleaned = str_cleaned.replace('\\', '')
    str_cleaned = str_cleaned.replace('—', '')
    str_cleaned = str_cleaned.replace('-', '')
    str_cleaned = str_cleaned.replace('...', '')
    str_cleaned = str_cleaned.replace('…', '')
    str_cleaned = str_cleaned.replace('END', '')
    str_cleaned = re.sub(r"\[(.*?)\]", '', str_cleaned) #remove all that is contained within brackets
    str_cleaned = re.sub(r"\((.*?)\)", '', str_cleaned) #remove all that is contained within parentheses
    str_cleaned = re.sub(r'\“(.+?)\”', '', str_cleaned) #remove all that is in quatation as he probably quoting someone
    str_cleaned = str_cleaned.replace('‘', '')
    str_cleaned = [wrd.lower() for wrd in str_cleaned.split(' ') if wrd != '']
    str_cleaned = ' '.join(str_cleaned)

    return str_cleaned

In [713]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

all_stop_words = []
with open('Filipino_Stopwords.txt', 'r') as fl:
    for n in fl:
        all_stop_words.append(n.strip())

all_stop_words.extend(ENGLISH_STOP_WORDS)

def create_word_cloud(ser, stop_words=all_stop_words, ax=plt):
    full_txt = ''
    for txt in ser:
        full_txt += txt
    
    wc = WordCloud(stopwords=stop_words, background_color='white', colormap='Dark2', random_state=42)
    cleaned_temp = [word for word in full_txt.split(' ') if word not in all_stop_words]
    txt = ' '.join(cleaned_temp)
    wc.generate(txt)
    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')